## Merge game databases

### Steps
- pick a main dataset
- rename columns
- for each added dataset
    - rename columns
    - rename platforms
    - check column values for anomalies
    - merge dataset to main
    - join columns using priority functions

In [61]:
import pandas as pd
import numpy as np
import column_join as cj

In [62]:
default_columns = [
'name',
'filename',
'summary',
'platform',
'release_date',
'release_year',
'genres',
'developer',
'publisher',
'players',
'cooperative',
'rating', # range 1-100
'user_rating' # range 1-10
]

### Launchbox
Lets choose the Launchbox dataset as our main to which we will merge other datasets

In [63]:
launchbox = pd.read_csv('arrm/launchbox.csv')

In [64]:
launchbox.columns

Index(['N°', 'DatabaseID', 'Name', 'ReleaseYear', 'Overview', 'MaxPlayers',
       'ReleaseType', 'Cooperative', 'VideoURL', 'CommunityRating', 'Platform',
       'ESRB', 'CommunityRatingCount', 'Genres', 'Developer', 'Publisher',
       'WikipediaURL', 'AlternateName_Australia', 'AlternateName_Brazil',
       'AlternateName_China', 'AlternateName_Europe', 'AlternateName_France',
       'AlternateName_Germany', 'AlternateName_Japan', 'AlternateName_Korea',
       'AlternateName_NorthAmerica', 'AlternateName_Spain',
       'AlternateName_UnitedStates', 'AlternateName_World'],
      dtype='object')

In [65]:
# keep these columns
launchbox = launchbox[['Name', 'ReleaseYear', 'Overview', 'MaxPlayers',
                      'Cooperative', 'Platform', 'CommunityRating', 
                      'Genres', 'Developer', 'Publisher']]

In [66]:
launchbox.rename(columns={'Name':'name',
                          'ReleaseYear' : 'release_year',
                          'Overview' : 'summary',
                          'MaxPlayers' : 'players',
                          'Cooperative' : 'cooperative',
                          'Platform' : 'platform',
                          'CommunityRating' : 'user_rating',
                          'Genres' : 'genres',
                          'Developer' : 'developer',
                          'Publisher' : 'publisher'}, inplace=True)

In [67]:
launchbox.head()

,name,release_year,summary,players,cooperative,platform,user_rating,genres,developer,publisher
0,20th Century Video Almanac,1993.0,"In The Best of Our Century, we've taken multim...",1.0,0,3DO Interactive Multiplayer,3.279851,Education,The Software Toolworks,The Software Toolworks
1,3D Atlas,1994.0,The World Isn't Flat. Why Should Your Atlas Be...,1.0,0,3DO Interactive Multiplayer,3.500000,Education,Electronic Arts,Electronic Arts
2,3DO Action Pak,1995.0,This is a four-game compilation pack that cont...,1.0,0,3DO Interactive Multiplayer,3.706667,Action,3DO,3DO
3,3DO de Shiru Miru Asobu Nakajima Miyuki,NaN,NaN,1.0,0,3DO Interactive Multiplayer,3.666667,NaN,NaN,Pony Canyon
4,3DO Demo Disc Program,NaN,A white binder with blue silk-screened art. Th...,NaN,0,3DO Interactive Multiplayer,3.372727,NaN,NaN,NaN


In [68]:
# double the user rating so it would be between 1-10
launchbox['user_rating'] = launchbox['user_rating'] * 2

### Merge with all_games

In [69]:
all_games = pd.read_csv('arrm/all_games.csv')

In [70]:
all_games.head()

,name,platform,release_date,summary,meta_score,user_review
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,"November 23, 1998","As a young boy, Link is tricked by Ganondorf, ...",99,9.1
1,Tony Hawk's Pro Skater 2,PlayStation,"September 20, 2000",As most major publishers' development efforts ...,98,7.4
2,Grand Theft Auto IV,PlayStation 3,"April 29, 2008",[Metacritic's 2008 PS3 Game of the Year; Also ...,98,7.7
3,SoulCalibur,Dreamcast,"September 8, 1999","This is a tale of souls and swords, transcendi...",98,8.4
4,Grand Theft Auto IV,Xbox 360,"April 29, 2008",[Metacritic's 2008 Xbox 360 Game of the Year; ...,98,7.9


In [71]:
# in user_review column, replace 'tbd' with None
all_games['user_review'] = all_games['user_review'].replace('tbd', np.nan)

In [72]:
# sort for duplicate removal such that non-empty user_review comes first
all_games = all_games.sort_values(['name', 'platform', 'user_review'])

# remove duplicates, keep first
print(len(all_games))
all_games = all_games.drop_duplicates(subset=['name', 'platform'], keep='first').sort_values(['name', 'platform'])
print(len(all_games))

18800
18690


In [73]:
all_games.columns

Index(['name', 'platform', 'release_date', 'summary', 'meta_score',
       'user_review'],
      dtype='object')

In [74]:
# meta_score and user_review columns need to be renamed
all_games.rename(columns={'meta_score':'rating', 'user_review' : 'user_rating'}, inplace=True)

In [75]:
# convert release date to timestamp
all_games['release_date'] = pd.to_datetime(all_games['release_date'])


In [76]:
# check distinct platform values on all_games
all_games['platform'].unique()

array([' Switch', ' Xbox One', ' Xbox', ' Xbox 360', ' PC',
       ' PlayStation 4', ' PlayStation 2', ' Wii', ' DS',
       ' PlayStation 3', ' Nintendo 64', ' PlayStation', ' PSP',
       ' GameCube', ' Dreamcast', ' Game Boy Advance', ' 3DS', ' Wii U',
       ' PlayStation 5', ' PlayStation Vita', ' Xbox Series X', ' Stadia'],
      dtype=object)

In [77]:
# remove leading space for all_games platform values
all_games['platform'] = all_games['platform'].apply(lambda x: x.strip())
all_games['platform'].unique()

array(['Switch', 'Xbox One', 'Xbox', 'Xbox 360', 'PC', 'PlayStation 4',
       'PlayStation 2', 'Wii', 'DS', 'PlayStation 3', 'Nintendo 64',
       'PlayStation', 'PSP', 'GameCube', 'Dreamcast', 'Game Boy Advance',
       '3DS', 'Wii U', 'PlayStation 5', 'PlayStation Vita',
       'Xbox Series X', 'Stadia'], dtype=object)

In [78]:
# distinct platform values on launchbox
launchbox['platform'].unique()

array(['3DO Interactive Multiplayer', 'Acorn Electron', 'Amstrad CPC',
       'Android', 'Aamber Pegasus', 'Acorn Archimedes', 'Acorn Atom',
       'Amstrad GX4000', 'Apple II', 'Apple iOS', 'Apple Mac OS',
       'Arcade', 'APF Imagination Machine', 'Apogee BK-01', 'Apple IIGS',
       'Atari 2600', 'Atari 800', 'Atari ST', 'BBC Microcomputer System',
       'Atari 5200', 'Atari 7800', 'Atari Jaguar', 'Atari Jaguar CD',
       'Atari Lynx', 'Atari XEGS', 'Bally Astrocade', 'ColecoVision',
       'Commodore 64', 'Camputers Lynx', 'Casio Loopy', 'Casio PV-1000',
       'Coleco ADAM', 'Commodore 128', 'Commodore Amiga',
       'Commodore Amiga CD32', 'Commodore VIC-20',
       'Fujitsu FM Towns Marty', 'Fujitsu FM-7', 'GameWave',
       'GCE Vectrex', 'Hector HRX', 'Microsoft MSX', 'Commodore CDTV',
       'Commodore MAX Machine', 'Commodore PET', 'Commodore Plus 4',
       'Dragon 32/64', 'EACA EG2000 Colour Genie', 'Elektronika BK',
       'Emerson Arcadia 2001', 'Enterprise', 'Entex A

In [79]:
# platform rename map for all_games to match launchbox platform names
platform_rename_map = {
    'PlayStation': 'Sony Playstation',
    'PlayStation 2': 'Sony Playstation 2',
    'PlayStation 3': 'Sony Playstation 3',
    'PlayStation 4': 'Sony Playstation 4',
    'PlayStation 5': 'Sony Playstation 5',
    'PlayStation Vita': 'Sony Playstation Vita',
    'PSP': 'Sony PSP',
    'Dreamcast' : 'Sega Dreamcast VMU',
    'Xbox 360' : 'Microsoft Xbox 360',
    'Xbox' : 'Microsoft Xbox',
    'Xbox One' : 'Microsoft Xbox One',
    'Xbox Series X' : 'Microsoft Xbox Series X',
    'Wii' : 'Nintendo Wii',
    'Wii U' : 'Nintendo Wii U',
    'Switch' : 'Nintendo Switch',
    'GameCube' : 'Nintendo GameCube',
    'Game Boy Advance' : 'Nintendo Game Boy Advance',
    '3DS' : 'Nintendo 3DS',
    'DS' : 'Nintendo DS',
    'Stadia' : 'Google Stadia',
    'PC' : 'Windows'
}


In [80]:
all_games['platform'] = all_games['platform'].replace(platform_rename_map)

In [81]:
# lets try merging
merged_df = pd.merge(launchbox, all_games, on=['name', 'platform'], how='outer', validate="many_to_many")

In [82]:
merged_df.head()

,name,release_year,summary_x,players,cooperative,platform,user_rating_x,genres,developer,publisher,release_date,summary_y,rating,user_rating_y
0,20th Century Video Almanac,1993.0,"In The Best of Our Century, we've taken multim...",1.0,0.0,3DO Interactive Multiplayer,6.559701,Education,The Software Toolworks,The Software Toolworks,NaT,NaN,NaN,NaN
1,3D Atlas,1994.0,The World Isn't Flat. Why Should Your Atlas Be...,1.0,0.0,3DO Interactive Multiplayer,7.000000,Education,Electronic Arts,Electronic Arts,NaT,NaN,NaN,NaN
2,3DO Action Pak,1995.0,This is a four-game compilation pack that cont...,1.0,0.0,3DO Interactive Multiplayer,7.413333,Action,3DO,3DO,NaT,NaN,NaN,NaN
3,3DO de Shiru Miru Asobu Nakajima Miyuki,NaN,NaN,1.0,0.0,3DO Interactive Multiplayer,7.333333,NaN,NaN,Pony Canyon,NaT,NaN,NaN,NaN
4,3DO Demo Disc Program,NaN,A white binder with blue silk-screened art. Th...,NaN,0.0,3DO Interactive Multiplayer,6.745455,NaN,NaN,NaN,NaT,NaN,NaN,NaN


In [83]:
# create user_rating column from user_rating_x and user_rating_y with a preference for user_rating_y if it exists
merged_df['user_rating'] = merged_df['user_rating_y'].fillna(merged_df['user_rating_x'])

In [84]:
merged_df['summary'] = merged_df.apply(cj.prioritize_summary, args=['summary_x', 'summary_y'], axis=1)

In [85]:
# drop user_rating_x, user_rating_y, summary_x, summary_y
merged_df = merged_df.drop(['user_rating_x', 'user_rating_y','summary_x','summary_y'], axis=1)

In [86]:
merged_df.head()

,name,release_year,players,cooperative,platform,genres,developer,publisher,release_date,rating,user_rating,summary
0,20th Century Video Almanac,1993.0,1.0,0.0,3DO Interactive Multiplayer,Education,The Software Toolworks,The Software Toolworks,NaT,NaN,6.559701,"In The Best of Our Century, we've taken multim..."
1,3D Atlas,1994.0,1.0,0.0,3DO Interactive Multiplayer,Education,Electronic Arts,Electronic Arts,NaT,NaN,7.0,The World Isn't Flat. Why Should Your Atlas Be...
2,3DO Action Pak,1995.0,1.0,0.0,3DO Interactive Multiplayer,Action,3DO,3DO,NaT,NaN,7.413333,This is a four-game compilation pack that cont...
3,3DO de Shiru Miru Asobu Nakajima Miyuki,NaN,1.0,0.0,3DO Interactive Multiplayer,NaN,NaN,Pony Canyon,NaT,NaN,7.333333,None
4,3DO Demo Disc Program,NaN,NaN,0.0,3DO Interactive Multiplayer,NaN,NaN,NaN,NaT,NaN,6.745455,A white binder with blue silk-screened art. Th...


### GameTDB

In [87]:
gametdb = pd.read_csv('arrm/games_on_gametdb.csv')

In [88]:
gametdb.head()

,N°,gametdb_id,gametdb_type,gametdb_region,gametdb_languages,gametdb_title_de,gametdb_title_en,gametdb_title_es,gametdb_title_fr,gametdb_title_it,...,gametdb_players,gametdb_genre,gametdb_rom,gametdb_rom_sans_ext,gametdb_platform,gametdb_title_cn,gametdb_title_tw,gametdb_synopsis_cn,gametdb_synopsis_tw,gametdb_rom_cleaned
0,880253,A22J,3DS,NTSC-J,JP,Bokujou Monogatari Futago no Mura+,Bokujou Monogatari Futago no Mura+,NaN,Bokujou Monogatari Futago no Mura+,Bokujou Monogatari Futago no Mura+,...,1.0,NaN,Bokujou Monogatari Futago no Mura+ (Japan) (JA...,Bokujou Monogatari Futago no Mura+ (Japan) (JA),3DS,Bokujou Monogatari Futago no Mura+,Bokujou Monogatari Futago no Mura+,NaN,NaN,Bokujou Monogatari Futago no Mura+
1,880254,A2AE,3DS,NTSC-U,EN,Pokémon Ultra Sun,Pokémon Ultra Sun,NaN,Pokémon Ultra Sun,Pokémon Ultra Sun,...,1.0,"adventure,role-playing,action rpg",Pokémon Ultra Sun (USA) (EN).3ds,Pokémon Ultra Sun (USA) (EN),3DS,Pokémon Ultra Sun,Pokémon Ultra Sun,NaN,NaN,Pokémon Ultra Sun
2,880255,A2AJ,3DS,NTSC-J,JP,Pokémon Ultra Sun,Pokémon Ultra Sun,NaN,Pokémon Ultra Sun,Pokémon Ultra Sun,...,1.0,"adventure,role-playing,action rpg",Pokémon Ultra Sun (Japan) (JA).3ds,Pokémon Ultra Sun (Japan) (JA),3DS,Pokémon Ultra Sun,Pokémon Ultra Sun,NaN,NaN,Pokémon Ultra Sun
3,880256,A2AK,3DS,NTSC-K,KR,Pokémon Ultra Sun,Pokémon Ultra Sun,NaN,Pokémon Ultra Sun,Pokémon Ultra Sun,...,1.0,"role-playing,action rpg",Pokémon Ultra Sun (Korea) (KO).3ds,Pokémon Ultra Sun (Korea) (KO),3DS,Pokémon Ultra Sun,Pokémon Ultra Sun,NaN,NaN,Pokémon Ultra Sun
4,880257,A2AP,3DS,PAL,"EN,FR,DE,ES,IT",Pokémon Ultra Sun,Pokémon Ultra Sun,Pokémon UltraSol,Pokémon Ultra Sun,Pokémon Ultra Sun,...,1.0,"action,adventure,role-playing,fantasy,action rpg","Pokémon Ultra Sun (Europe) (EN,FR,DE,ES,IT).3ds","Pokémon Ultra Sun (Europe) (EN,FR,DE,ES,IT)",3DS,Pokémon Ultra Sun,Pokémon Ultra Sun,Return to Alola for an alternate adventure in ...,Return to Alola for an alternate adventure in ...,Pokémon Ultra Sun


In [89]:
gametdb.columns

Index(['N°', 'gametdb_id', 'gametdb_type', 'gametdb_region',
       'gametdb_languages', 'gametdb_title_de', 'gametdb_title_en',
       'gametdb_title_es', 'gametdb_title_fr', 'gametdb_title_it',
       'gametdb_title_ja', 'gametdb_title_ko', 'gametdb_title_nl',
       'gametdb_title_pt', 'gametdb_title_ru', 'gametdb_synopsis_de',
       'gametdb_synopsis_en', 'gametdb_synopsis_es', 'gametdb_synopsis_fr',
       'gametdb_synopsis_it', 'gametdb_synopsis_ja', 'gametdb_synopsis_ko',
       'gametdb_synopsis_nl', 'gametdb_synopsis_pt', 'gametdb_synopsis_ru',
       'gametdb_developer', 'gametdb_publisher', 'gametdb_date',
       'gametdb_players', 'gametdb_genre', 'gametdb_rom',
       'gametdb_rom_sans_ext', 'gametdb_platform', 'gametdb_title_cn',
       'gametdb_title_tw', 'gametdb_synopsis_cn', 'gametdb_synopsis_tw',
       'gametdb_rom_cleaned'],
      dtype='object')

In [90]:
gametdb.rename(columns={'gametdb_platform' : 'platform', 
                        'gametdb_title_en' : 'name',
                        'gametdb_synopsis_en' : 'summary', 
                        'gametdb_developer' : 'developer',
                        'gametdb_publisher' : 'publisher', 
                        'gametdb_date' : 'release_date', 
                        'gametdb_players' : 'players',
                        'gametdb_genre' : 'genres'
                        }, inplace=True)

In [91]:
gametdb = gametdb[['platform', 'name', 'summary', 
                   'developer', 'publisher', 'release_date', 
                   'players', 'genres']]

In [92]:
# convert release_date to timestamp
gametdb['release_date'] = pd.to_datetime(gametdb['release_date'])

In [93]:
# drop where 'name' is missing
gametdb = gametdb.dropna(subset=['name'])

In [94]:
# check name platform duplicates
duplicates = gametdb[gametdb.duplicated(subset=['name', 'platform'], keep=False)].sort_values(['name', 'platform', 'summary', 'developer', 'publisher'])
duplicates.head()

,platform,name,summary,developer,publisher,release_date,players,genres
33635,Switch,#Funtime,NaN,One Guy Games,The Quantum Astrophysicists Guild,1980-01-01,1.0,"music,action,arcade"
33636,Switch,#Funtime,NaN,One Guy Games,The Quantum Astrophysicists Guild,1980-01-01,1.0,"action,music,arcade,shooter"
16490,PS3,&: Sora no Mukou de Saki Masuyou ni,NaN,NaN,NaN,2014-01-01,1.0,NaN
21026,PS3,&: Sora no Mukou de Saki Masuyou ni,NaN,NaN,NaN,2014-01-01,1.0,NaN
48495,WiiU,(Event Preview) Extreme Exorcism,NaN,Golden Ruby Games,Ripstone,2018-01-01,1.0,"action,platformer,arcade"


In [95]:
# sort for duplicate removal such that the non-empty value for each column comes first
gametdb = gametdb.sort_values(['name', 'platform', 'summary', 'developer', 'publisher'])

# remove duplicates, keep first
print(len(gametdb))
gametdb = gametdb.drop_duplicates(subset=['name', 'platform'], keep='first').sort_values(['name', 'platform'])
print(len(gametdb))

49099
34463


In [96]:
merged_df['platform'].unique()

array(['3DO Interactive Multiplayer', 'Acorn Electron', 'Amstrad CPC',
       'Android', 'Aamber Pegasus', 'Acorn Archimedes', 'Acorn Atom',
       'Amstrad GX4000', 'Apple II', 'Apple iOS', 'Apple Mac OS',
       'Arcade', 'APF Imagination Machine', 'Apogee BK-01', 'Apple IIGS',
       'Atari 2600', 'Atari 800', 'Atari ST', 'BBC Microcomputer System',
       'Atari 5200', 'Atari 7800', 'Atari Jaguar', 'Atari Jaguar CD',
       'Atari Lynx', 'Atari XEGS', 'Bally Astrocade', 'ColecoVision',
       'Commodore 64', 'Camputers Lynx', 'Casio Loopy', 'Casio PV-1000',
       'Coleco ADAM', 'Commodore 128', 'Commodore Amiga',
       'Commodore Amiga CD32', 'Commodore VIC-20',
       'Fujitsu FM Towns Marty', 'Fujitsu FM-7', 'GameWave',
       'GCE Vectrex', 'Hector HRX', 'Microsoft MSX', 'Commodore CDTV',
       'Commodore MAX Machine', 'Commodore PET', 'Commodore Plus 4',
       'Dragon 32/64', 'EACA EG2000 Colour Genie', 'Elektronika BK',
       'Emerson Arcadia 2001', 'Enterprise', 'Entex A

In [97]:
# check platform unique values
gametdb['platform'].unique()

array(['Switch', 'PS3', 'Wii', 'DS', 'WiiU', '3DS'], dtype=object)

In [98]:
platform_rename_map = {
    '3DS' : 'Nintendo 3DS',
    'DS' : 'Nintendo DS',
    'PS3' : 'Sony Playstation 3',
    'Switch' : 'Nintendo Switch',
    'Wii' : 'Nintendo Wii', 
    'WiiU' : 'Nintendo Wii U'
}

In [99]:
gametdb['platform'] = gametdb['platform'].replace(platform_rename_map)

In [100]:
gametdb.head()

,platform,name,summary,developer,publisher,release_date,players,genres
28366,Nintendo Switch,#1 Anagrams,NaN,NaN,Eclipse Games,2021-05-14,1.0,"board game,puzzle"
29808,Nintendo Switch,#1 Anagrams Sudokus Bundle,NaN,NaN,Eclipse Games,2022-02-25,1.0,"puzzle,board game"
27818,Nintendo Switch,#1 Crosswords,NaN,NaN,Eclipse Games,2021-02-19,1.0,puzzle
28000,Nintendo Switch,#1 Crosswords Bundle,NaN,NaN,Eclipse Games,2021-06-25,1.0,"board game,puzzle"
28001,Nintendo Switch,#1 Crosswords Sudokus Bundle,NaN,NaN,Eclipse Games,1980-01-01,1.0,"puzzle,board game"


#### Merging gametdb with main

In [101]:
# lets try merging
merged_df = pd.merge(merged_df, gametdb, on=['name', 'platform'], how='outer', validate="many_to_many")

In [102]:
merged_df = merged_df.replace({np.nan : None})

In [103]:
merged_df.columns

Index(['name', 'release_year', 'players_x', 'cooperative', 'platform',
       'genres_x', 'developer_x', 'publisher_x', 'release_date_x', 'rating',
       'user_rating', 'summary_x', 'summary_y', 'developer_y', 'publisher_y',
       'release_date_y', 'players_y', 'genres_y'],
      dtype='object')

In [104]:
merged_df['release_date'] = merged_df.apply(cj.prioritize_value, args=['release_date_x', 'release_date_y'], axis=1)
merged_df['players'] = merged_df.apply(cj.prioritize_value, args=['players_x', 'players_y'], axis=1)
merged_df['genres'] = merged_df.apply(cj.join_genres, args=['genres_x', 'genres_y'], axis=1)
merged_df['developer'] = merged_df.apply(cj.prioritize_columns, args=['developer_x', 'developer_y'], axis=1)
merged_df['publisher'] = merged_df.apply(cj.prioritize_columns, args=['publisher_x', 'publisher_y'], axis=1)
merged_df['summary'] = merged_df.apply(cj.prioritize_summary, args=['summary_x', 'summary_y'], axis=1)


In [105]:
# drop user_rating_x, user_rating_y, summary_x, summary_y
merged_df = merged_df.drop(['players_x', 'players_y',
                            'summary_x','summary_y',
                            'developer_x', 'developer_y',
                            'publisher_x', 'publisher_y',
                            'release_date_x', 'release_date_y',
                            'genres_x', 'genres_y'], axis=1)

### DAT DOS dataset

In [106]:
dat_dos = pd.read_csv('arrm/dat_database_dos.csv')

In [107]:
dat_dos.head()

,ID,Name,Developer,Y,Genre,Description,Publisher
0,61191,Stunt Island,The Assembly Line,19920101T000000,"Flight Simulator,Vehicle Simulation",Stunt Island was marketed as The Stunt Flying ...,"Walt Disney Computer Software, Inc."
1,61192,Tommy's Yahtzee,Tommy's Toys,19860101T000000,Board / Party Game,Tommy's Yahtzee is a shareware implementation ...,Freeware
2,61193,Fantasy Empires,Silicon Knights,19931001T000000,Strategy,Build and control an Empire! In Fantasy Empire...,"Strategic Simulations, Inc."
3,61194,Pune,Hardware Not Included,19950101T000000,Action,Pune is a light cycle/snake-type game for mult...,Hardware Not Included
4,61195,Mortal Kombat,Probe Software Ltd.,19940525T000000,"Action,Fighting","Five Hundred years ago, an ancient and well re...","Acclaim Entertainment, Inc."


In [108]:
dat_dos.isnull().sum()

ID              0
Name            0
Developer       3
Y               0
Genre           1
Description    12
Publisher      31
dtype: int64

In [109]:
dat_dos['platform'] = 'MS-DOS'

In [110]:
# columns need to be renamed
dat_dos.rename(columns={'Name':'name', 
                        'Developer' : 'developer', 
                        'Y' : 'release_date', 
                        'Genre' : 'genres',
                        'Description' : 'summary',
                        'Publisher' : 'publisher'}, inplace=True)

In [111]:
# drop ID column
dat_dos = dat_dos.drop(['ID'], axis=1)

In [112]:
duplicates = dat_dos[dat_dos.duplicated(subset=['name', 'platform'], keep=False)].sort_values(['name', 'platform', 'summary', 'developer', 'publisher'])
duplicates.head()

,name,developer,release_date,genres,summary,publisher,platform
1178,Action Fighter,"Softstar Entertainment, Inc.",19940101T000000,"Action,Fighting",A side scrolling beat em' up from Korea.,"Softstar Entertainment, Inc.",MS-DOS
7245,Action Fighter,"Softstar Entertainment, Inc.",19940101T000000,"Action,Fighting",A side scrolling beat em' up from Korea.,"Softstar Entertainment, Inc.",MS-DOS
6155,Action Fighter,Sega Enterprises Ltd.,19890101T000000,"Action,Racing / Driving",This is a top down vertically scrolling shoote...,Firebird Software,MS-DOS
7326,Action Fighter,Sega Enterprises Ltd.,19890101T000000,"Action,Racing / Driving",This is a top down vertically scrolling shoote...,Firebird Software,MS-DOS
5794,Aldo's Assault,Unknown,19910101T000000,"Action,Platform",Aldo's Assault is the third game in the Aldo s...,Freeware,MS-DOS


In [113]:
# sort for duplicate removal such that the non-empty value for each column comes first
dat_dos = dat_dos.sort_values(['name', 'platform', 'summary', 'developer', 'publisher'])

# remove duplicates, keep first
print(len(dat_dos))
dat_dos = dat_dos.drop_duplicates(subset=['name', 'platform'], keep='first').sort_values(['name', 'platform'])
print(len(dat_dos))

7482
7082


In [114]:
# there are some dashes in release_date, replace them with None
dat_dos['release_date'].replace('-', None, inplace=True)

# release_date to datetime
dat_dos['release_date'] = pd.to_datetime(dat_dos['release_date'])

In [115]:
# this one was easy. ready for merging
merged_df = pd.merge(merged_df, dat_dos, on=['name', 'platform'], how='outer', validate="many_to_many")

In [116]:
merged_df.head()

,name,release_year,cooperative,platform,rating,user_rating,release_date_x,players,genres_x,developer_x,publisher_x,summary_x,developer_y,release_date_y,genres_y,summary_y,publisher_y
0,20th Century Video Almanac,1993.0,0.0,3DO Interactive Multiplayer,None,6.559701,NaT,1.0,Education,The Software Toolworks,The Software Toolworks,"In The Best of Our Century, we've taken multim...",NaN,NaT,NaN,NaN,NaN
1,3D Atlas,1994.0,0.0,3DO Interactive Multiplayer,None,7.0,NaT,1.0,Education,Electronic Arts,Electronic Arts,The World Isn't Flat. Why Should Your Atlas Be...,NaN,NaT,NaN,NaN,NaN
2,3DO Action Pak,1995.0,0.0,3DO Interactive Multiplayer,None,7.413333,NaT,1.0,Action,3DO,3DO,This is a four-game compilation pack that cont...,NaN,NaT,NaN,NaN,NaN
3,3DO de Shiru Miru Asobu Nakajima Miyuki,None,0.0,3DO Interactive Multiplayer,None,7.333333,NaT,1.0,None,None,Pony Canyon,None,NaN,NaT,NaN,NaN,NaN
4,3DO Demo Disc Program,None,0.0,3DO Interactive Multiplayer,None,6.745455,NaT,NaN,None,None,None,A white binder with blue silk-screened art. Th...,NaN,NaT,NaN,NaN,NaN


In [117]:
# replace  so the code in the next cell would work
merged_df = merged_df.replace({np.nan : None})

In [118]:
# perform column joins using functions on release_date, genres, developer, publisher and summary
merged_df['release_date'] = merged_df.apply(cj.prioritize_value, args=['release_date_x', 'release_date_y'], axis=1)
merged_df['genres'] = merged_df.apply(cj.join_genres, args=['genres_x', 'genres_y'], axis=1)
merged_df['developer'] = merged_df.apply(cj.prioritize_columns, args=['developer_x', 'developer_y'], axis=1)
merged_df['publisher'] = merged_df.apply(cj.prioritize_columns, args=['publisher_x', 'publisher_y'], axis=1)
merged_df['summary'] = merged_df.apply(cj.prioritize_summary, args=['summary_x', 'summary_y'], axis=1)

In [119]:
# drop user_rating_x, user_rating_y, summary_x, summary_y
merged_df = merged_df.drop(['summary_x','summary_y',
                            'developer_x', 'developer_y',
                            'publisher_x', 'publisher_y',
                            'release_date_x', 'release_date_y',
                            'genres_x', 'genres_y'], axis=1)

In [120]:
merged_df.head()

,name,release_year,cooperative,platform,rating,user_rating,players,release_date,genres,developer,publisher,summary
0,20th Century Video Almanac,1993.0,0.0,3DO Interactive Multiplayer,None,6.559701,1.0,NaT,Education,The Software Toolworks,The Software Toolworks,"In The Best of Our Century, we've taken multim..."
1,3D Atlas,1994.0,0.0,3DO Interactive Multiplayer,None,7.0,1.0,NaT,Education,Electronic Arts,Electronic Arts,The World Isn't Flat. Why Should Your Atlas Be...
2,3DO Action Pak,1995.0,0.0,3DO Interactive Multiplayer,None,7.413333,1.0,NaT,Action,3DO,3DO,This is a four-game compilation pack that cont...
3,3DO de Shiru Miru Asobu Nakajima Miyuki,None,0.0,3DO Interactive Multiplayer,None,7.333333,1.0,NaT,None,None,Pony Canyon,None
4,3DO Demo Disc Program,None,0.0,3DO Interactive Multiplayer,None,6.745455,None,NaT,None,None,None,A white binder with blue silk-screened art. Th...


## DAT MAME

In [121]:
dat_mame = pd.read_csv('arrm/dat_database_mame.csv')

/tmp/ipykernel_1544335/409855410.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dat_mame = pd.read_csv('arrm/dat_database_mame.csv')


In [122]:
dat_mame.head()

,ID,systemname_mame,name,filename,cloneof_mame,developer,release_date,genre
0,355065,advmame 0.94-RetroPie-260.dat,"PuckMan (Japan set 1, Probably Bootleg)",puckman,NaN,Namco,1980/01/01,NaN
1,355066,advmame 0.94-RetroPie-260.dat,PuckMan (Japan set 2),puckmana,puckman,Namco,1980/01/01,NaN
2,355067,advmame 0.94-RetroPie-260.dat,PuckMan (Japan set 1 with speedup hack),puckmanf,puckman,Namco,1980/01/01,NaN
3,355068,advmame 0.94-RetroPie-260.dat,Puckman (Falcom),puckmanh,puckman,hack,1980/01/01,NaN
4,355069,advmame 0.94-RetroPie-260.dat,Pac-Man (Midway),pacman,puckman,[Namco] (Midway license),1980/01/01,NaN


In [123]:
dat_mame.isnull().sum()

ID                      0
systemname_mame         0
name                    0
filename                0
cloneof_mame        88733
developer           11564
release_date        94049
genre              193224
dtype: int64

In [124]:
dat_mame['systemname_mame'].value_counts()

systemname_mame
MAME 0.260.dat                                   46199
MAME 0.240 (Arcade).dat                          36604
mame2016 - MAME 0.174 Arcade XML.dat             32071
mame2015 - MAME 0.160 XML.dat                    30906
Mame 0.239.xml                                   29027
mame2010.xml                                      8833
FinalBurn_Neo_ClrMame_Pro_XML_Arcade_only.dat     6998
advmame 12-106.dat                                6166
advmame 0.94-RetroPie-260.dat                     5563
mame2003-plus.xml                                 5219
mame2003.xml                                      4726
vgmplay.xml                                       3645
FB Alpha 2012 v0.2.97.30.dat                      3369
mame2000 mame4all - MAME 0.37b5.dat               2241
FB Alpha v0.2.96.71 working_roms.dat               684
Name: count, dtype: int64

In [125]:
# looks like duplicates for each version of MAME. Let's use the one that says Arcade.
dat_mame_subset = dat_mame[dat_mame['systemname_mame'] == 'MAME 0.240 (Arcade).dat']

In [126]:
dat_mame_subset.head()

,ID,systemname_mame,name,filename,cloneof_mame,developer,release_date,genre
51807,406872,MAME 0.240 (Arcade).dat,005,005,NaN,Sega,1981/01/01,NaN
51808,406873,MAME 0.240 (Arcade).dat,"100 Lions (10219211, NSW/ACT)",100lions,NaN,Aristocrat,2006/01/01,NaN
51809,406874,MAME 0.240 (Arcade).dat,"10-Yard Fight (World, set 1)",10yard,NaN,Irem,1983/01/01,NaN
51810,406875,MAME 0.240 (Arcade).dat,"10-Yard Fight '85 (US, Taito license)",10yard85,10yard,Irem (Taito license),1985/01/01,NaN
51811,406876,MAME 0.240 (Arcade).dat,10-Yard Fight (Japan),10yardj,10yard,Irem,1983/01/01,NaN


In [127]:
# rename genre to genres
dat_mame_subset.rename(columns={'genre' : 'genres'}, inplace=True)

# only desired columns
dat_mame_subset = dat_mame_subset[['name', 'filename', 'developer', 'release_date', 'genres']]

# create platform column named Arcade
dat_mame_subset['platform'] = 'Arcade'

# release_date to datetime
dat_mame_subset['release_date'] = pd.to_datetime(dat_mame_subset['release_date'])

/tmp/ipykernel_1544335/2486768337.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat_mame_subset.rename(columns={'genre' : 'genres'}, inplace=True)


In [128]:
duplicates = dat_mame_subset[dat_mame_subset.duplicated(subset=['name', 'platform'], keep=False)].sort_values(['name', 'platform', 'developer', 'release_date', 'genres'])
duplicates.head()

,name,filename,developer,release_date,genres,platform
59175,Galaxy Wars (Taito),galxwarst2,Universal (Taito Corporation license),1979-01-01,NaN,Arcade
59174,Galaxy Wars (Taito),galxwarst,Universal (Taito license),1979-01-01,NaN,Arcade
69255,"Road Hog (Barcrest) (MPU4) (RR6 1.2, hack)",m4rhog_h12,hack,1991-01-01,NaN,Arcade
69256,"Road Hog (Barcrest) (MPU4) (RR6 1.2, hack)",m4rhog_h13,hack,1991-01-01,NaN,Arcade


In [129]:
# sort for duplicate removal such that the non-empty value for each column comes first
dat_mame_subset = dat_mame_subset.sort_values(['name', 'platform', 'developer', 'release_date', 'genres'])

# remove duplicates, keep first
print(len(dat_mame_subset))
dat_mame_subset = dat_mame_subset.drop_duplicates(subset=['name', 'platform'], keep='first').sort_values(['name', 'platform'])
print(len(dat_mame_subset))

36604
36602


In [130]:
# merging
merged_df = pd.merge(merged_df, dat_mame_subset, on=['name', 'platform'], how='outer', validate="many_to_many")

In [131]:
merged_df = merged_df.replace({np.nan : None})

# column join functions for release_date, developer and genres
merged_df['release_date'] = merged_df.apply(cj.prioritize_value, args=['release_date_x', 'release_date_y'], axis=1)
merged_df['developer'] = merged_df.apply(cj.prioritize_columns, args=['developer_x', 'developer_y'], axis=1)
merged_df['genres'] = merged_df.apply(cj.join_genres, args=['genres_x', 'genres_y'], axis=1)

# drop processed columns
merged_df = merged_df.drop(['developer_x', 'developer_y',
                            'genres_x', 'genres_y',
                            'release_date_x', 'release_date_y'], axis=1)

In [132]:
merged_df.head()

,name,release_year,cooperative,platform,rating,user_rating,players,publisher,summary,filename,release_date,developer,genres
0,20th Century Video Almanac,1993.0,0.0,3DO Interactive Multiplayer,None,6.559701,1.0,The Software Toolworks,"In The Best of Our Century, we've taken multim...",None,NaT,The Software Toolworks,Education
1,3D Atlas,1994.0,0.0,3DO Interactive Multiplayer,None,7.0,1.0,Electronic Arts,The World Isn't Flat. Why Should Your Atlas Be...,None,NaT,Electronic Arts,Education
2,3DO Action Pak,1995.0,0.0,3DO Interactive Multiplayer,None,7.413333,1.0,3DO,This is a four-game compilation pack that cont...,None,NaT,3DO,Action
3,3DO de Shiru Miru Asobu Nakajima Miyuki,None,0.0,3DO Interactive Multiplayer,None,7.333333,1.0,Pony Canyon,None,None,NaT,None,None
4,3DO Demo Disc Program,None,0.0,3DO Interactive Multiplayer,None,6.745455,None,None,A white binder with blue silk-screened art. Th...,None,NaT,None,None


### Recalbox

In [133]:
recalbox = pd.read_csv('arrm/recalbox_gamelist.csv')
recalbox_xml = pd.read_csv('arrm/recalbox_gamelist_xml.csv')

In [134]:
recalbox.columns

Index(['numauto_rom', 'nomjeu_rom', 'fichier_rom', 'description_rom',
       'image_rom', 'rating_rom', 'annee_rom', 'developer_rom',
       'publisher_rom', 'genre_rom', 'players_rom', 'cache_rom', 'favoris_rom',
       'boxart_rom', 'screenshot_rom', 'wheel_rom', 'video_rom', 'mix_rom',
       'playcount_rom', 'lastplayed_rom', 'md5_rom', 'core_rom',
       'emulator_rom', 'cartridge_rom', 'hash_rom', 'manual_rom', 'region_rom',
       'thumbnail_rom', 'marquee_rom', 'genreid_rom', 'fanart_rom', 'map_rom',
       'titleshot_rom', 'lang_rom', 'gameid_rom', 'kid_rom', 'adult_rom',
       'arcadesystemname_rom', 'gametime_rom', 'boxback_rom', 'temporary_rom',
       'bezel_rom', 'ratio_rom', 'rotation_rom', 'extra1_rom', 'famille_rom',
       'mode_rom'],
      dtype='object')

In [135]:
# checking if ready for concatenation
recalbox_xml.columns == recalbox.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [136]:
# append dataframes
recalbox = pd.concat([recalbox, recalbox_xml])

In [137]:
recalbox.head()

,numauto_rom,nomjeu_rom,fichier_rom,description_rom,image_rom,rating_rom,annee_rom,developer_rom,publisher_rom,genre_rom,...,arcadesystemname_rom,gametime_rom,boxback_rom,temporary_rom,bezel_rom,ratio_rom,rotation_rom,extra1_rom,famille_rom,mode_rom
0,17605,10000000-in-1 [p1][!],./10000000-in-1 [p1][!].zip,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17606,10-Yard Fight,./10-Yard Fight (U).zip,10-Yard Fight is a simple arcade game based on...,./miximages/10-Yard Fight (U).png,0.4,19851206T000000,Irem,Nintendo,Sports,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17607,10-Yard Fight,./10-Yard Fight.zip,10-Yard Fight is a simple arcade game based on...,./miximages/10-Yard Fight.png,0.4,19851206T000000,Irem,Nintendo,Sports,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17608,118-in-1 [p1][!],./118-in-1 [p1][!].zip,NaN,./miximages/118-in-1 [p1][!].png,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17609,11-in-1 Ball Games [p1][!],./11-in-1 Ball Games [p1][!].zip,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
# keep only those columns
recalbox = recalbox.iloc[:, 1:11]

# drop image column
recalbox = recalbox.drop(['image_rom'], axis=1)

In [139]:
recalbox.rename(columns={'nomjeu_rom':'name', 
                         'fichier_rom':'filename',
                         'description_rom':'summary', 
                         'rating_rom':'user_rating', 
                         'annee_rom':'release_date',
                         'developer_rom':'developer', 
                         'publisher_rom':'publisher',
                         'genre_rom':'genres', 
                         'players_rom':'players'}, inplace=True)

In [140]:
recalbox['platform'] = 'Nintendo Entertainment System'

# release_date to datetime
recalbox['release_date'] = pd.to_datetime(recalbox['release_date'])

# deal w nans
recalbox = recalbox.replace({np.nan : None})

In [141]:
duplicates = recalbox[recalbox.duplicated(subset=['name', 'filename'], keep=False)].sort_values(['name', 'filename', 'summary', 'developer', 'publisher'])
duplicates.head()

,name,filename,summary,user_rating,release_date,developer,publisher,genres,players,platform
1,10-Yard Fight,./10-Yard Fight (U).zip,10-Yard Fight is a simple arcade game based on...,0.4,1985-12-06 00:00:00,Irem,Nintendo,Sports,1-2,Nintendo Entertainment System
2,10-Yard Fight,./10-Yard Fight (U).zip,10-Yard Fight is a simple arcade game based on...,0.4,1985-12-06 00:00:00,Irem,Nintendo,Sports,1-2,Nintendo Entertainment System
2,10-Yard Fight,./10-Yard Fight.zip,10-Yard Fight is a simple arcade game based on...,0.4,1985-12-06 00:00:00,Irem,Nintendo,Sports,1-2,Nintendo Entertainment System
1,10-Yard Fight,./10-Yard Fight.zip,10-Yard Fight is a simple arcade game based on...,0.4,1985-12-06 00:00:00,Irem,Nintendo,Sports,1-2,Nintendo Entertainment System
3,118-in-1 [p1][!],./118-in-1 [p1][!].zip,None,None,None,None,None,None,None,Nintendo Entertainment System


In [142]:
# sort for duplicate removal such that the non-empty value for each column comes first
recalbox = recalbox.sort_values(['name', 'filename', 'summary', 'developer', 'publisher'])

# remove duplicates, keep first
print(len(recalbox))
recalbox = recalbox.drop_duplicates(subset=['name'], keep='first').sort_values(['name', 'platform'])
print(len(recalbox))

3990
1545


In [143]:
# code for formatting players values to our standard
def extract_max_players(players):
    if players is None:
        return None
    if '-' in str(players):
        return int(players.split('-')[-1])  # Extract the maximum number of players after splitting by '-'
    else:
        return int(players)  # If it's already a single value, return it as an integer

In [144]:
recalbox['players'] = recalbox['players'].apply(extract_max_players)

In [145]:
# user rating is 0-1 so multiply by 10 so it'd be bevt 1-10
recalbox['user_rating'] = recalbox['user_rating'] * 10

In [146]:
recalbox.head()

,name,filename,summary,user_rating,release_date,developer,publisher,genres,players,platform
1,10-Yard Fight,./10-Yard Fight (U).zip,10-Yard Fight is a simple arcade game based on...,4.0,1985-12-06 00:00:00,Irem,Nintendo,Sports,2.0,Nintendo Entertainment System
0,10000000-in-1 [p1][!],./10000000-in-1 [p1][!].zip,None,NaN,None,None,None,None,NaN,Nintendo Entertainment System
4,11-in-1 Ball Games [p1][!],./11-in-1 Ball Games [p1][!].zip,None,NaN,None,None,None,None,NaN,Nintendo Entertainment System
3,118-in-1 [p1][!],./118-in-1 [p1][!].zip,None,NaN,None,None,None,None,NaN,Nintendo Entertainment System
5,150-in-1 (Mapper 202) [p1][!],./150-in-1 (Mapper 202) [p1][!].zip,None,NaN,None,None,None,None,NaN,Nintendo Entertainment System


In [147]:
# merging
merged_df = pd.merge(merged_df, recalbox, on=['name', 'platform'], how='outer', validate="many_to_many")

In [148]:
merged_df.head()

,name,release_year,cooperative,platform,rating,user_rating_x,players_x,publisher_x,summary_x,filename_x,...,developer_x,genres_x,filename_y,summary_y,user_rating_y,release_date_y,developer_y,publisher_y,genres_y,players_y
0,20th Century Video Almanac,1993.0,0.0,3DO Interactive Multiplayer,None,6.559701,1.0,The Software Toolworks,"In The Best of Our Century, we've taken multim...",None,...,The Software Toolworks,Education,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3D Atlas,1994.0,0.0,3DO Interactive Multiplayer,None,7.0,1.0,Electronic Arts,The World Isn't Flat. Why Should Your Atlas Be...,None,...,Electronic Arts,Education,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3DO Action Pak,1995.0,0.0,3DO Interactive Multiplayer,None,7.413333,1.0,3DO,This is a four-game compilation pack that cont...,None,...,3DO,Action,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3DO de Shiru Miru Asobu Nakajima Miyuki,None,0.0,3DO Interactive Multiplayer,None,7.333333,1.0,Pony Canyon,None,None,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3DO Demo Disc Program,None,0.0,3DO Interactive Multiplayer,None,6.745455,None,None,A white binder with blue silk-screened art. Th...,None,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [149]:
merged_df = merged_df.replace({np.nan : None})

In [150]:
# column join functions for rating, players, publisher, developer, summary, release_date
merged_df['user_rating'] = merged_df.apply(cj.prioritize_columns, args=['user_rating_x', 'user_rating_y'], axis=1)
merged_df['players'] = merged_df.apply(cj.prioritize_value, args=['players_x', 'players_y'], axis=1)
merged_df['publisher'] = merged_df.apply(cj.prioritize_columns, args=['publisher_x', 'publisher_y'], axis=1)
merged_df['developer'] = merged_df.apply(cj.prioritize_columns, args=['developer_x', 'developer_y'], axis=1)
merged_df['summary'] = merged_df.apply(cj.prioritize_summary, args=['summary_x', 'summary_y'], axis=1)
merged_df['release_date'] = merged_df.apply(cj.prioritize_value, args=['release_date_x', 'release_date_y'], axis=1)
merged_df['filename'] = merged_df.apply(cj.prioritize_columns, args=['filename_x', 'filename_y'], axis=1)
merged_df['genres'] = merged_df.apply(cj.join_genres, args=['genres_x', 'genres_y'], axis=1)

In [151]:
# drop processed columns
merged_df = merged_df.drop(['developer_x', 'developer_y',
                            'publisher_x', 'publisher_y',
                            'release_date_x', 'release_date_y',
                            'genres_x', 'genres_y',
                            'filename_x', 'filename_y',
                            'user_rating_x', 'user_rating_y',
                            'players_x', 'players_y',
                            'summary_x', 'summary_y'], axis=1)

In [152]:
# drop duplicates
merged_df = merged_df.drop_duplicates()

In [153]:
merged_df.head()

,name,release_year,cooperative,platform,rating,user_rating,players,publisher,developer,summary,release_date,filename,genres
0,20th Century Video Almanac,1993.0,0.0,3DO Interactive Multiplayer,None,6.559701,1.0,The Software Toolworks,The Software Toolworks,"In The Best of Our Century, we've taken multim...",NaT,None,Education
1,3D Atlas,1994.0,0.0,3DO Interactive Multiplayer,None,7.0,1.0,Electronic Arts,Electronic Arts,The World Isn't Flat. Why Should Your Atlas Be...,NaT,None,Education
2,3DO Action Pak,1995.0,0.0,3DO Interactive Multiplayer,None,7.413333,1.0,3DO,3DO,This is a four-game compilation pack that cont...,NaT,None,Action
3,3DO de Shiru Miru Asobu Nakajima Miyuki,None,0.0,3DO Interactive Multiplayer,None,7.333333,1.0,Pony Canyon,None,None,NaT,None,None
4,3DO Demo Disc Program,None,0.0,3DO Interactive Multiplayer,None,6.745455,NaN,None,None,A white binder with blue silk-screened art. Th...,NaT,None,None


In [154]:
import pickle
with open('merged_df.pkl', 'wb') as f:
    pickle.dump(merged_df, f)